In [1]:
import csv
import re
import os
import tqdm
import shutil

In [2]:
output_dir = 'output'
our_output_dir = 'our_output'
our_data_dir = 'our_data'
metadata_dir = 'metadata'
project_model_data_dir = '/vol/main/initial-data-point-selection-backup/src/models/data'

experiment_output_dirs = os.listdir(output_dir)
experiment_output_dirs 

['results_data_processed_isic_dino_v2.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_matek_swav_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_cifar10_simclr_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_isic_simclr_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_cifar10_swav_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_retinopathy_swav_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_matek_simclr_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_cifar10_dino_v2.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_jurkat_swav_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_jurkat_dino_v2.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_matek_dino_v2.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_retinopathy_simclr_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_jurkat_simclr_v1.pickle_tsne_kmediods_csd_200_0',
 'results_data_processed_isic_swav_

In [3]:
def generate_our_output(indices_path, features_path, our_output_path):
    names = []
    with open(features_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            names.append(row['name'])

    # indices
    indices = []
    with open(indices_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            indices.append(int(row['indices']))

    # names
    with open(our_output_path, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)

        csv_writer.writerow(['name'])
        for i in indices:
            csv_writer.writerow([names[i]])

In [6]:
def process_experiment_output_dir(experiment_output_dir):
    dataset, model_type, model_version = re.search(
        r'results_data_processed_([^_]+)_([^_]+)_([^_]+)\.', 
        experiment_output_dir
    ).groups()

    features_path = os.path.join(our_data_dir, f'{dataset}_{model_type}_{model_version}.csv')
    our_output_dir = os.path.join(project_model_data_dir, dataset, model_type, model_version, 'cold_paws', '200')
    os.makedirs(our_output_dir, exist_ok=True)
    metadata_path = os.path.join(metadata_dir, f'{dataset}_{model_type}_{model_version}.csv')

    # saving metadata
    shutil.copy(metadata_path, os.path.join(our_output_dir, 'metadata.json'))

    for i in range(5):
        indices_path = os.path.join(output_dir, experiment_output_dir, f'X200-{i}-indicies.csv') 
        our_output_path = os.path.join(our_output_dir, f'{i}.csv')

        generate_our_output(indices_path, features_path, our_output_path)

In [7]:
for experiment_output_dir in tqdm.tqdm(experiment_output_dirs):
    process_experiment_output_dir(experiment_output_dir)

100%|██████████| 14/14 [03:04<00:00, 13.19s/it]
